In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import kampuan as kp

kp.test()

'Hello World!'

In [33]:

phrase='สวัสดี'
kp.tokenize(phrase)

['สวัส', 'ดี']

In [34]:
kp.tokenize('ตลาด')

['ตลาด']

* 2 word 2 syllabus case


In [100]:
case_1 ={'กินข้าว':'กาวขิ้น',
         'หิวจัง':'หังจิว',
        'คำผวน':'ควนผำ',
        'นอนแล้ว':'แนวร้อน',
        'ตะปู':'ตูปะ',
        'นักเรียน':'เนียนรัก',
        'ขนม':'ขมหน่ะ'}

tmp=[kp.tokenize(key) for key in case_1.keys()]
tmp

# get 

[['กิน', 'ข้าว'],
 ['หิว', 'จัง'],
 ['คำ', 'ผวน'],
 ['นอน', 'แล้ว'],
 ['ตะ', 'ปู'],
 ['นัก', 'เรียน'],
 ['ขนม']]

In [55]:
# ref
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float
    lat: float

@dataclass
class Capital(Position):
    country: str
from enum import Enum

class Color(Enum):
    red = 1
    green = 2
    blue = 3
    
@dataclass
class Test:
    _name: str="schbell"

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, v: str) -> None:
        self._name = v


# Subwords

* initial_consonant
    * single อักษรเดี่ยว
        * sound_level (low/mid/high)
        * low_single, low_double
    * cluster อักษรควบ
        * true_cluster อักษรควบแท้
        * fasle_cluster อักษรควบไม่แท้
    * leading_cluster อักษรนำ–อักษรตาม
        * hornum_leading หอ นำ
        * oarnum_leading ออ นำ
        * non_conform_leading นำด้วยสูง หรือ กลาง  นคร ขนม
* final_consonant
    * type (dead/live)
    * sound (eng)
        * k: กก
        * ng: กง
        * t: กด
        * m: กม
        * n: กน
        * p: กบ
        * y: เกย
        * v: เกอว
    * sound_level low/mid/high
* vowel
* tone http://www.thai-language.com/ref/tone-rules
    * tone_mark
    * tone_sound

In [204]:
REP

'_'

In [305]:
# Read Asset
import pythainlp
import pandas as pd

df_init_con =pd.read_excel('../src/kampuan/assets/initial_consonants.xlsx',skiprows=1).fillna(method='ffill')
df_fin_con =pd.read_excel('../src/kampuan/assets/final_consonants.xlsx',skiprows=1)
df_fin_con['sound']=df_fin_con['sound'].fillna(method='ffill')
df_vowel_form =pd.read_excel('../src/kampuan/assets/thai_vowel_forms.xlsx')

THAI_CHARS =pythainlp.thai_characters
THAI_CONS =pythainlp.thai_consonants
THAI_VOW =pythainlp.thai_vowels
THAI_TONE=pythainlp.thai_tonemarks

In [343]:
# Vowel extract
def get_vowel_form(REP:str='[REP]',is_sort=True):
    vowel_list=df_vowel_form['รูปสระ'].str.replace('\xa0','').str.replace('-',REP).str.strip().tolist()
    vowel_list.append(REP)
    if is_sort:
        vowel_list.sort(key=len,reverse=True)
    return vowel_list

def get_vowel_pattern():
    REP='[REP]'
    vowel_pattern=[]
    vowel_forms=get_vowel_form(REP=REP)
    for vow in vowel_forms:
        if vow.find(REP):
            vowel_pattern.append(f"({vow.replace(REP,')(.{1,2})(')})(.*)")
        else:
            vowel_pattern.append(f"{vow.replace(REP,'(.{1,2})(')})(.*)")
            
    return vowel_pattern

def remove_tone_mark(text,tone_marks=THAI_TONE):
    for mark in tone_marks:
        text=text.replace(mark,'')
    return text

vowel_pattern=get_vowel_pattern()

In [344]:
pattern

def get_pattern(pattern,text):
    m=re.match(pattern,text)
    return m

In [352]:
import re


def get_pattern(pattern,text):
    m=re.match(pattern,text)
    return m

def match_vowel_form(text):
    for i,pattern in enumerate(vowel_pattern):
        m=re.match(pattern,text)
        if m:
            return (i,m,pattern)
    
    return (-1,text,text)
        
vowel_forms=get_vowel_form(REP='-')
test_text =['เขียว',
            'เกรียน',
            'ตู่',
            'ตด',
            'กร',
            'นอน',
           'ขนุน',
           'จัง',
           'กำ',
           'ใหญ่',
            'ใคร',
           'เดา',
           'เปรต',
           'เสร็จ',
           'ปีน',
           'เปี๊ยะ',
           'คั่ว',
           'แก้ม',
           'แข็ง',
           'เกลอ']

for text in test_text:
    print(text)
    plain=remove_tone_mark(text)
    (i,m,pattern)=match_vowel_form(plain)   
    print('===')
    print(m.groups(),vowel_forms[i])

เขียว
===
('เ', 'ข', 'ีย', 'ว') เ-ีย
เกรียน
===
('เ', 'กร', 'ีย', 'น') เ-ีย
ตู่
===
('ต', 'ู', '') -ู
ตด
===
('ตด', '', '') -
กร
===
('กร', '', '') -
นอน
===
('น', 'อ', 'น') -อ
ขนุน
===
('ขน', 'ุ', 'น') -ุ
จัง
===
('จ', 'ั', 'ง') -ั
กำ
===
('ก', 'ำ', '') -ำ
ใหญ่
===
('ใ', 'หญ', '', '') ใ-
ใคร
===
('ใ', 'คร', '', '') ใ-
เดา
===
('เ', 'ด', 'า', '') เ-า
เปรต
===
('เ', 'ปร', '', 'ต') เ-
เสร็จ
===
('เ', 'สร', '็', 'จ') เ-็
ปีน
===
('ป', 'ี', 'น') -ี
เปี๊ยะ
===
('เ', 'ป', 'ียะ', '') เ-ียะ
คั่ว
===
('ค', 'ัว', '') -ัว
แก้ม
===
('แ', 'กม', '', '') แ-
แข็ง
===
('แ', 'ข', '็', 'ง') แ-็


In [273]:
import re

def get_pattern(pattern,text):
    m=re.match(pattern,text)
    return m

for text in ['เขียว','เกรียน','ตู่','ตด']:
    print(get_pattern(vowel_pattern,text))

<re.Match object; span=(0, 1), match='เ'>
<re.Match object; span=(0, 1), match='เ'>
None
None


In [195]:
m.group('word')

'ข'

In [93]:
pythainlp.thai_tonemarks

'่้๊๋'

In [72]:
pythainlp.thai_punctuations

'๏๚๛'

In [77]:
['-'+char for char in pythainlp.thai_below_vowels]

['-ุ', '-ู']

In [94]:
['_'+char for char in pythainlp.thai_tonemarks]

['_่', '_้', '_๊', '_๋']

In [79]:
['_'+char for char in pythainlp.thai_above_vowels]

['_ั', '_ิ', '_ี', '_ึ', '_ื', '_ํ', '_็']

In [82]:
[char+'_' for char in pythainlp.thai_lead_vowels]

['เ_', 'แ_', 'โ_', 'ใ_', 'ไ_']

In [80]:
['_'+char for char in pythainlp.thai_follow_vowels]

['_ะ', '_า', '_ำ', '_ๅ']

In [102]:
pythainlp.thai_vowels

'ฤฦะัาำิีึืุูเแโใไๅํ็'

In [97]:
import pythainlp
from dataclasses import dataclass


@dataclass
class ThaiSubWord:
    def __init__(self, word:str='เกี๊ยว'):
        # Character base
        self._raw: str=word
        self._vowels:List[str]=self.vowels
        self._consonants:List[str]=self.consonants
        self._tone_mark:List[str]=self.tone_mark
        
        # Type based
        self._inti_con:str =None
        self._final_con:str=None
        self._vowel_form:str=None
        self._tone:str=None
    
    @property
    def raw(self):
        return self._raw
    
    @property
    def vowels(self):
        return [char for char in self._raw if char in THAI_VOW]
    
    @property
    def consonants(self):
        return [char for char in self._raw if char in THAI_CONS]
    
    @property
    def tone_mark(self):
        return [char for char in self._raw if char in THAI_TONE]
    
        

a=ThaiSubWord()

a.__dict__

{'_raw': 'เกี๊ยว',
 '_vowels': ['เ', 'ี'],
 '_consonants': ['ก', 'ย', 'ว'],
 '_tone_mark': ['๊'],
 '_inti_con': None,
 '_final_con': None,
 '_vowel_form': None,
 '_tone': None}

http://www.thai-language.com/ref/tone-rules

# Tien's

In [9]:
from kampuan.util.extract import thai_subwords

In [25]:
x=[thai_subwords(orig_word=syll) for syll in kp.puan_kam(phrase)]
x[0].start



'หว'

In [ ]:
class thai_match_pattern():
    def __init__(self, start_regex, vowel_regex, end_regex, sound_regex):
        self.start_regex = start_regex
        self.vowel_regex = vowel_regex
        self.end_regex = end_regex
        self.sound_regex = sound_regex
        
    def match_start(self, orig_word):        
        matches = re.findall(self.start_regex, orig_word, re.DOTALL)
        return matches
    

In [ ]:
#Reference https://www.geeksforgeeks.org/longest-common-substring-dp-29/
# Returns length of longest common  
# substring of X[0..m-1] and Y[0..n-1]  
def LCSubStr(X, Y, m, n): 
      
    # Create a table to store lengths of 
    # longest common suffixes of substrings.  
    # Note that LCSuff[i][j] contains the  
    # length of longest common suffix of  
    # X[0...i-1] and Y[0...j-1]. The first 
    # row and first column entries have no 
    # logical meaning, they are used only 
    # for simplicity of the program. 
      
    # LCSuff is the table with zero  
    # value initially in each cell 
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)] 
      
    # To store the length of  
    # longest common substring 
    result = 0 
  
    # Following steps to build 
    # LCSuff[m+1][n+1] in bottom up fashion 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if (i == 0 or j == 0): 
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]): 
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j]) 
            else: 
                LCSuff[i][j] = 0
    return result 

In [34]:
import re
class thai_subwords():
    """Design to take input as single Thai token or word and extract start, vowel, end and sound of said token
    For additional data regarding different Thai words types and pronunciations, 
    consult:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn"""
    def __init__(self, orig_word, debug=False):
        self.orig_word = orig_word #Try not to change this value
        self._start = ''
        self._vowel = ''
        self._end = ''
        self._sound = ''
        self._double_start_sound = False #used to flag with that has two starting alphabets with sound like 'ตลาด = ตะ ลาด'
        self._high_lead_low = False #case อักษรสูงนำอักษรต่ำ
        self.debug = debug
        
        #Need to deal with negative lookahead case for word like "เกรียน" to correctly extract กร
        self.vowels_form = ['ะ', ' ั' ' ็', 'า', ' ิ' , ' ่' , " ่" , ' ํ', ' ุ', ' ู', 'เ', 'ใ', 'ไ', 'โ', 'อ', 'ย', 'ว', 'ฤ', 'ฤๅ', 'ฦ', 'ฦๅ']
        combine_vowel_regex_1 = '(?!เ.{1,4}ี.*ย)(?!เ.{1,4}ี.*ยะ)(?!เ.{1,4}ื.*อ)(?!เ.{1,4}ื.*อะ)(?!เ.{1,4}.*อะ)(?!เ.{1,4}.*อ)(?!เ.{1,4}.*าะ)'
        combine_vowel_regex_2 = '(?!เ.{1,4}.*า)(?!แ.{1,4}.*า)(?!.{1,4}ัวะ)(?!โ.{1,4}.*ะ)'
        combine_vowel_regex = combine_vowel_regex_1 + combine_vowel_regex_2
        self.start_regex_1 = ['(.{1,4})ะ','(.{1,4})า', '(.{1,4})ิ.*', '(.{1,4})ี.*', '(.{1,4})ึ', '(.{1,4})ื.*', '(.{1,4})ุ.*', '(.{1,4})ู.*', 'เ(.{1,4})็', 
                            'เ(.{1,4}).*', '(.{1,4}).*อ','แ(.{1,4}).*', '(.{1,4})ั.*', 'โ(.{1,4}).*', '(.{1,4}).*ำ', 'ใ(.{1,4}).*', 'ไ(.{1,4})']
        self.start_regex_1  = [combine_vowel_regex+regex for regex in self.start_regex_1]
        self.start_regex_2 = ['เ(.{1,4})ี.*ยะ', 'เ(.{1,4})ี.*ย', 'เ(.{1,4})ื.*อะ', 'เ(.{1,4})ื.*อ', 'เ(.{1,4}).*อ', 'เ(.{1,4}).*อะ',
                              'เ(.{1,4}).*า',
                              'เ(.{1,4}).*าะ','แ(.{1,4}).*ะ','(.{1,4})ัวะ','โ(.{1,4}).*ะ', 'เ(.{1,4})็', 'เ(.{1,4})ิ.*']
        self.end_regex_1 = ['\[REP\]า(.{1,4})', '\[REP\]ิ(.{1,4})', '\[REP\]ี(.{1,4})', '\[REP\]ึ(.{1,4})', '\[REP\]ือ(.{1,4})', 
                            '\[REP\]ุ(.{1,4})', '\[REP\]ู(.{1,4})', 'เ\[REP\]็(.{1,4})', 'เ\[REP\](.{1,4})', '\[REP\]อ(.{1,4})',
                            'แ\[REP\](.{1,4})', '\[REP\]ั(.{1,4})', 'โ\[REP\](.{1,4})','\[REP\]ำ(.{1,4})', 'ใ\[REP\](.{1,4})', 
                            'ไ\[REP\](.{1,4})']
        self.end_regex_2 = ['เ\[REP\]ีย(.{1,4})', 'เ\[REP\]ือ(.{1,4})', 'เ\[REP\]อ(.{1,4})', 'เ\[REP\]อะ(.{1,4})','เ\[REP\]า(.{1,4})']
        
        self.start_regex  = self.start_regex_1 + self.start_regex_2 
        self.end_regex  = self.end_regex_1 + self.end_regex_2 
        #self.vowel_regex = vowel_regex
        #self.end_regex = end_regex
        self.sound_regex = ['(่)', '(้)', '.(๊)','()๋']
        
        #initialize the special character cases
        #คำควบแท้
        self.two_char_combine = ['กร','กล', 'กว', 'คร', 'ขร', 'คล', 'ขล','คว', 'ขว', 'ตร', 'ปร', 'ปล', 'พร', 'พล' ,'ผล',
                   'บร','บล','ดร','ฟร','ฟล','ทร','จร','ซร','ปร','สร']
        #คำนำ
        self.lead_char_nosound = ['อย','หง','หญ','หน','หม','หย', 'หร','หล','หว']
        #อักษรสูงนำอักษรต่ำ
        self._lead_char_high_low = ['ขน', 'ขม','สม','สย','สน','ขย','ฝร','ถล','ผว','ตน','จม','ตล',]
        #อักษรสูงนำอักษรกลาง
        self._high_char_high_medium = ['ผท','ผด','ผก','ผอ','ผช']
        
        
    # using property decorator 
    @property
    def start(self): 
        #print("getter method called") 
        temp_start = self.match_pattern(self.orig_word, self.start_regex)
        if len(temp_start)==1: return temp_start
        else: 
            return self.check_double_start_alphabets(temp_start)
        #return self.match_pattern(self.orig_word, self.start_regex)
    
    @property
    def sound(self):
        if self.match_pattern(self.orig_word, self.sound_regex) is None: return ''
        return self.match_pattern(self.orig_word, self.sound_regex)
    
    @property
    def vowel(self):
        return self.orig_word.replace(self.sound,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
#     def vowel(self):
#         return self.orig_word.replace(self.sound,'').replace(self.end,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
    
    @property #might not need this one in final application as we need vowel
    def end(self):
        if self.match_pattern(self.vowel, self.end_regex) not in self.vowels_form: 
            return self.match_pattern(self.vowel, self.end_regex)
        else:
            return None

#     # a setter function 
#     @start.setter 
#     def start(self): 
#         print("setter method called") 
#         self._start = match_pattern(self.start_regex)
    
    def match_pattern(self, text, regex_pattern):
        output_start = None
        if self.debug: print(text)
        for pattern in regex_pattern:
            matches = re.search(pattern, text, re.DOTALL)
            
            if matches is not None and self.debug: print(pattern, matches.group(1))
            if matches is not None: output_start = matches.group(1)
        return output_start
    
    def check_double_start_alphabets(self, text):
        '''Use to check in case starting alphabet is longer than one, 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        two_char = self.lead_char_nosound + self.two_char_combine
        
        max_lcs = 0
        if text in two_char: return text
        else:
            for char in two_char:
                if LCSubStr(text, char, len(text), len(char)) > max_lcs: 
                    max_lcs = LCSubStr(text, char, len(text), len(char))
                    max_lcs_char = char
            return max_lcs_char
        
        return None
    
    def check_double_start_sound(self, text):
        '''Use to address the case where start alphabets are double and need to split to reflect actual pronunciation 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        
        return None
    
    def remove_karan(self, regex_pattern):
        return None
        #Write specific function for removing ์